In [37]:
# Import necessary libraries
from pydantic import BaseModel, EmailStr, HttpUrl, Json, field_validator
from typing import List, Dict, Optional
import re
from enum import Enum


In [38]:


class UserModel(BaseModel):
    name: str
    email: str

    @field_validator('email')
    def validate_email(cls, v):
        email_regex = r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"
        if not re.match(email_regex, v):
            raise ValueError('value is not a valid email address')
        return v

user = UserModel(name="John Doe", email="john.doe@example.com")
print(user)
print(user.model_dump())
print(user.name, user.email,sep='\n')

name='John Doe' email='john.doe@example.com'
{'name': 'John Doe', 'email': 'john.doe@example.com'}
John Doe
john.doe@example.com


In [39]:
from pydantic import BaseModel, HttpUrl, Json, ValidationError, validator
from typing import Dict, List, Optional
from enum import Enum
import re

# Define an Enum for colors
class Color(str, Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"

# Define a nested model for Address
class Address(BaseModel):
    street: str
    city: str

# Define a model with various types of fields
class UserModel(BaseModel):
    name: str
    age: int
    email: str
    website: HttpUrl
    metadata: Json
    color: Color
    address: Address
    scores: Dict[str, int]
    hobbies: List[str]
    is_active: Optional[bool] = True

    @field_validator('age')
    def check_age(cls, v):
        if v < 18:
            raise ValueError('Age must be 18 or older')
        return v
    
    @field_validator('email')
    def validate_email(cls, v):
        email_regex = r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"
        if not re.match(email_regex, v):
            raise ValueError('value is not a valid email address')
        return v
import json
try:
    user = UserModel(
        name="John Doe",
        age=25,
        email="john.doe@example.com",
        website="https://www.example.com",
        metadata=json.dumps({"key": "value"}), 
        color=Color.BLUE,
        address=Address(street="123 Main St", city="Anytown"),
        scores={"math": 90, "english": 85},
        hobbies=["reading", "hiking"]
    )
except ValidationError as e:
    print(e)



In [40]:
print(user)

# Access fields
print(user.name)
print(user.age)
print(user.email)
print(user.website)
print(user.metadata)
print(user.color)
print(user.address)
print(user.scores)
print(user.hobbies)
print(user.is_active)


name='John Doe' age=25 email='john.doe@example.com' website=Url('https://www.example.com/') metadata={'key': 'value'} color=<Color.BLUE: 'blue'> address=Address(street='123 Main St', city='Anytown') scores={'math': 90, 'english': 85} hobbies=['reading', 'hiking'] is_active=True
John Doe
25
john.doe@example.com
https://www.example.com/
{'key': 'value'}
Color.BLUE
street='123 Main St' city='Anytown'
{'math': 90, 'english': 85}
['reading', 'hiking']
True


In [41]:


# Modify fields
user.name = "Jane Doe"
user.age = 30
user.email = "jane.doe@example.com"
user.website = r"https://www.example.com/jane"
user.metadata = {"key": "new_value"}
user.color = Color.GREEN
user.address.street = "456 Elm St"
user.scores["math"] = 95
user.hobbies.append("cooking")
user.is_active = False

# Print the modified user instance
print(user.model_dump_json(indent=2))


{
  "name": "Jane Doe",
  "age": 30,
  "email": "jane.doe@example.com",
  "website": "https://www.example.com/jane",
  "metadata": {
    "key": "new_value"
  },
  "color": "green",
  "address": {
    "street": "456 Elm St",
    "city": "Anytown"
  },
  "scores": {
    "math": 95,
    "english": 85
  },
  "hobbies": [
    "reading",
    "hiking",
    "cooking"
  ],
  "is_active": false
}


In [42]:
from pydantic import BaseModel, validator
from typing import List, Optional

class Address(BaseModel):
    street: str
    city: str
    zip_code: Optional[str] = None # Make zip_code truly optional

class User(BaseModel):
    name: str
    age: int
    email: str
    addresses: List[Address]

    @field_validator('email')
    def validate_email(cls, v):
        if '@' not in v:
            raise ValueError('must contain an @ symbol')
        return v

    @field_validator('age')
    def check_age(cls, v):
        if v < 18:
            raise ValueError('Age must be 18 or older')
        return v

    @validator('addresses', each_item=True)
    def check_address(cls, v):
        if not v.street or not v.city:
            raise ValueError('Each address must have a street and city')
        return v

# Example usage
user_data = {
    "name": "John Doe",
    "age": 30,
    "email": "john.doe@example.com",
    "addresses": [
        {"street": "123 Main St", "city": "Anytown"},
        {"street": "456 Elm St", "city": "Anytown", "zip_code": "12345"}
    ]
}

user = User(**user_data)
print(user.model_dump_json(indent=2))


{
  "name": "John Doe",
  "age": 30,
  "email": "john.doe@example.com",
  "addresses": [
    {
      "street": "123 Main St",
      "city": "Anytown",
      "zip_code": null
    },
    {
      "street": "456 Elm St",
      "city": "Anytown",
      "zip_code": "12345"
    }
  ]
}


C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_19188\318601836.py:27: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator('addresses', each_item=True)


In [43]:
from pydantic import BaseModel, validator
from typing import List, Optional

class Address(BaseModel):
    street: str
    city: str
    zip_code: Optional[str]

class User(BaseModel):
    name: str
    age: int
    email: str
    addresses: List[Address]

    @validator('email')
    def validate_email(cls, v):
        if '@' not in v:
            raise ValueError('must contain an @ symbol')
        return v

    @validator('age')
    def check_age(cls, v):
        if v < 18:
            raise ValueError('Age must be 18 or older')
        return v

    @validator('addresses', each_item=True)
    def check_address(cls, v):
        if not v.street or not v.city:
            raise ValueError('Each address must have a street and city')
        return v

# Example usage
user_data = {
    "name": "John Doe",
    "age": 30,
    "email": "john.doe@example.com",
    "addresses": [
        {"street": "123 Main St", "city": "Anytown"},
        {"street": "456 Elm St", "city": "Anytown", "zip_code": "12345"}
    ]
}

user = User(**user_data)
print(user)


C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_19188\1917132016.py:15: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator('email')
C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_19188\1917132016.py:21: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator('age')
C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_19188\1917132016.py:27: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You

ValidationError: 1 validation error for User
addresses.0.zip_code
  Field required [type=missing, input_value={'street': '123 Main St', 'city': 'Anytown'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing